In [1]:
import os
import numpy as np
from osgeo import gdal, osr
import pandas as pd

In [2]:
import geoio

In [3]:
BASE_DIR = '..'
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [4]:
import sys
sys.path.append(BASE_DIR)
from funcs import create_space

In [5]:
'''
The goal of each of these functions is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, cons_pc

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = 'consumption_aggregate/ihs4 consumption aggregate.csv'
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'household_geovariables/householdgeovariablesihs4.csv'
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'

    # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 215.182
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['case_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', 'HHID', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='case_id')
    df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'mw'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_ethiopia():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2016', 'LSMS')
    consumption_file = 'Consumption Aggregate/cons_agg_w3.csv'
    consumption_pc_col = 'total_cons_ann' # per capita
    hhsize_col = 'hh_size' # people in household

    geovariables_file = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
    lat_col = 'lat_dd_mod'
    lon_col = 'lon_dd_mod'

    # purchasing power parity for ethiopia in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=ET)
    ppp = 7.882
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['household_id2', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['household_id2', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='household_id2')
    df_combined.drop(['household_id2'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'eth'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_nigeria():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'nigeria_2016', 'LSMS')
    consumption_file = 'cons_agg_wave3_visit1.csv'
    consumption_pc_col = 'totcons' # per capita
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'nga_householdgeovars_y3.csv'
    lat_col = 'LAT_DD_MOD'
    lon_col = 'LON_DD_MOD'

    # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
    ppp = 95.255
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['hhid', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['hhid', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='hhid')
    df_combined.drop(['hhid'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'ng'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

'''
The goal of each of these functions is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, cons_pc

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2019', 'LSMS')
    consumption_file = 'ihs5_consumption_aggregate.csv'
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'householdgeovariables_ihs5.csv'
    lat_col = 'ea_lat_mod'
    lon_col = 'ea_lon_mod'

    # purchasing power parity for malawi in 2019 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 285.2
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['case_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='case_id')
    df_combined.drop(['case_id'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'mw'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [6]:
df_mw = process_malawi()

<ipython-input-5-310860a28dce>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [7]:
df_eth = process_ethiopia()
df_ng = process_nigeria()

<ipython-input-5-310860a28dce>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
<ipython-input-5-310860a28dce>:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [10]:
df_mw.shape, df_eth.shape, df_ng.shape

((780, 4), (523, 4), (664, 4))

In [13]:
df_ng

,country,cluster_lat,cluster_lon,cons_pc
0,ng,4.315786,6.268753,4.317717
1,ng,4.328719,6.308172,4.880711
2,ng,4.398427,7.183962,8.767258
3,ng,4.425192,7.166935,10.774504
4,ng,4.619377,7.684946,5.191333
...,...,...,...,...
659,ng,13.202805,7.721468,1.165778
660,ng,13.339897,5.214480,2.710443
661,ng,13.555542,6.246647,1.759571
662,ng,13.604251,5.741676,1.647506


In [14]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [15]:
tifs[0]

Class Name        : GeoImage
Driver Name       : GTiff
Data Type         : Float32
File Name         : ..\data/nightlights/viirs_2015_00N060W.tif
File List         : ['..\\data/nightlights/viirs_2015_00N060W.tif']
Dimensions        : (1, 28800, 15600) (nlayers, nrows, ncols)
Resolution        : (0.0041666667, 0.0041666667) (x,y)
Extent            : (-60.00208333335, 0.00208333335, 59.99791762665001,
                    -64.99791718665) (ul_x, ul_y, lr_x, lr_y)
Projection String : GEOGCS["WGS 84",
                     DATUM["WGS_1984",
                         SPHEROID["WGS 84",6378137,298.257223563]],
                     PRIMEM["Greenwich",0],
                     UNIT["degree",0.0174532925199433,
                         AUTHORITY["EPSG","9122"]],
                     AXIS["Latitude",NORTH],
                     AXIS["Longitude",EAST]]
Geo Transform     : (-60.00208333335, 0.0041666667, 0.0, 0.00208333335, 0.0,
                    -0.0041666667)
Authority         : EPSG:9122

In [16]:
tifs[1]

Class Name        : GeoImage
Driver Name       : GTiff
Data Type         : Float32
File Name         : ..\data/nightlights/viirs_2015_75N060W.tif
File List         : ['..\\data/nightlights/viirs_2015_75N060W.tif']
Dimensions        : (1, 28800, 18000) (nlayers, nrows, ncols)
Resolution        : (0.0041666667, 0.0041666667) (x,y)
Extent            : (-60.00208333335, 75.00208333335, 59.99791762665001,
                    0.0020827333499937595) (ul_x, ul_y, lr_x, lr_y)
Projection String : GEOGCS["WGS 84",
                     DATUM["WGS_1984",
                         SPHEROID["WGS 84",6378137,298.257223563]],
                     PRIMEM["Greenwich",0],
                     UNIT["degree",0.0174532925199433,
                         AUTHORITY["EPSG","9122"]],
                     AXIS["Latitude",NORTH],
                     AXIS["Longitude",EAST]]
Geo Transform     : (-60.00208333335, 0.0041666667, 0.0, 75.00208333335, 0.0,
                    -0.0041666667)
Authority         : EPSG:9122

In [17]:
tif_array = np.squeeze(tifs[0].get_data())

In [15]:
tif_array.shape

(15600, 28800)

In [18]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [19]:
add_nightlights(df_mw, tifs[0], tif_array)

In [20]:
df_mw

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295
...,...,...,...,...,...
775,mw,-9.591378,33.057450,1.409932,0.000000
776,mw,-9.550397,33.291558,1.242801,0.000000
777,mw,-9.519230,33.139193,1.804122,0.003557
778,mw,-9.507538,33.259649,1.791725,0.000000


In [21]:
del tif_array
import gc
gc.collect()

0

In [22]:
tif_array = np.squeeze(tifs[1].get_data())

In [23]:
add_nightlights(df_eth, tifs[1], tif_array)
add_nightlights(df_ng, tifs[1], tif_array)

In [24]:
df_eth['nightlights'].mean()

0.6727544

In [25]:
df_mw.corr()

<ipython-input-25-6428fa58dd1e>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_mw.corr()


,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [26]:
for country in ['malawi_2016', 'ethiopia_2016', 'nigeria_2016']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [27]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed/clusters.csv'), index=False)

In [29]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2016', 'processed/clusters.csv'), index=False)

In [31]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2016', 'processed/clusters.csv'), index=False)